In [1]:
import src_lead_reporting as ld
import datascience as ds

In [2]:
lead=ld.lead_reporting_persona()

In [3]:
lead.create_leads_table_RS()

Success
Success
The Main persona lead table is created: na_field_marketing.persona_leads_analysis 


In [3]:
df=lead.calculations(start_year=2016,push_to_RS=True,table_name='python_lead_analysis')

--------- Lead Report for Business Analyst for the year 2016 ---------
Total number of leads: 11288
Accepted Leads: 2983
Converted Leads: 142
--------- Lead Report for Business Analyst for the year 2017 ---------
Total number of leads: 11587
Accepted Leads: 3885
Converted Leads: 190
--------- Lead Report for Business Analyst for the year 2018 ---------
Total number of leads: 16499
Accepted Leads: 6527
Converted Leads: 196
--------- Lead Report for Business Analyst for the year 2019 ---------
Total number of leads: 20562
Accepted Leads: 10062
Converted Leads: 212
--------- Lead Report for Business Analyst for the year 2020 ---------
Total number of leads: 14187
Accepted Leads: 5210
Converted Leads: 89


--------- Lead Report for IT Manager for the year 2016 ---------
Total number of leads: 30915
Accepted Leads: 8315
Converted Leads: 479
--------- Lead Report for IT Manager for the year 2017 ---------
Total number of leads: 23432
Accepted Leads: 7896
Converted Leads: 405
--------- Lead R